In [76]:
import os
from google.cloud import bigquery
import pandas as pd

In [77]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "DemoBigQuery.json"

In [78]:
client = bigquery.Client()

In [79]:
def table_bigqery_to_dataframe(sql_query=""" """):
    # Jalankan query
    query_job = client.query(sql_query)
    print("Query job started...")
    print("Waiting for query to complete...")

    # Tunggu hingga query selesai
    results = query_job.result()
    print("Query completed successfully.")

    # Konversi hasil query ke DataFrame
    df = results.to_dataframe()

    # Tampilkan DataFrame
    # print(df.head(100))  # Menampilkan 10 baris pertama dari DataFrame
    return df

In [80]:
jawaban1 = table_bigqery_to_dataframe(sql_query=""" 
WITH
  agregasi AS (
  SELECT
    DISTINCT(user_id) AS id,
    CASE
      WHEN SUM(sale_price) < 50 THEN 'New Customer'
      WHEN SUM(sale_price) BETWEEN 50
    AND 200 THEN 'Regular Customer'
      WHEN SUM(sale_price) BETWEEN 200 AND 500 THEN 'Loyal Customer'
      WHEN SUM(sale_price) > 500 THEN 'VIP Customer'
  END
    AS Kategori_Customer,
    SUM(sale_price) AS Total_Order_Value
  FROM
    `bigquery-public-data.thelook_ecommerce.order_items`
  GROUP BY
    id)

SELECT
  Kategori_Customer,
  COUNT(id) AS Jumlah_Customer,
  AVG(Total_Order_Value) AS Rata2_Order_Value
FROM	
  agregasi
GROUP BY
  Kategori_Customer

""")
jawaban1

Query job started...
Waiting for query to complete...
Query completed successfully.


/home/mahaputra971/anaconda3/envs/project_digital_skola/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Kategori_Customer,Jumlah_Customer,Rata2_Order_Value
0,Loyal Customer,15032,300.375284
1,Regular Customer,38279,109.571722
2,New Customer,24473,28.830038
3,VIP Customer,2001,681.977741


In [81]:
jawaban2 = table_bigqery_to_dataframe(sql_query=""" 
SELECT
    product_id,
    SUM(sale_price) AS total_revenue,
    COUNT(*) AS total_quantity,
    AVG(sale_price) AS avg_sale_price,
    RANK() OVER (ORDER BY SUM(sale_price) DESC) AS revenue_rank
FROM
    bigquery-public-data.thelook_ecommerce.order_items
WHERE
    status = 'Complete'  -- hanya hitung yang berhasil terkirim
GROUP BY
    product_id
ORDER BY
    total_revenue DESC;

""")
jawaban2.head(100)


Query job started...
Waiting for query to complete...
Query completed successfully.


/home/mahaputra971/anaconda3/envs/project_digital_skola/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,product_id,total_revenue,total_quantity,avg_sale_price,revenue_rank
0,24447,4995.000000,5,999.000000,1
1,24201,4515.000000,5,903.000000,2
2,24428,3612.000000,4,903.000000,3
3,2796,2709.000000,3,903.000000,4
4,24195,2574.000000,6,429.000000,5
...,...,...,...,...,...
95,24208,1049.850037,3,349.950012,96
96,19640,1044.000000,3,348.000000,97
97,21483,1043.000031,5,208.600006,98
98,20884,1040.000000,4,260.000000,99


In [84]:
jawaban3 = table_bigqery_to_dataframe(sql_query="""
WITH regional_data AS (
    -- Langkah 1: Gabungkan data pengguna, pesanan, dan item pesanan untuk negara US
    SELECT
        u.state,
        u.id AS user_id,
        o.order_id,
        oi.sale_price
    FROM
        `bigquery-public-data.thelook_ecommerce.users` AS u
    JOIN
        `bigquery-public-data.thelook_ecommerce.orders` AS o ON u.id = o.user_id
    JOIN
        `bigquery-public-data.thelook_ecommerce.order_items` AS oi ON o.order_id = oi.order_id
    WHERE
        u.country = 'United States'
        AND o.status IN ('Complete', 'Shipped') -- Menganalisis pesanan yang sudah selesai atau dikirim
),
state_performance AS (
    -- Langkah 2: Hitung metrik performa untuk setiap negara bagian (state)
    SELECT
        state,
        COUNT(DISTINCT user_id) AS total_customers,
        COUNT(DISTINCT order_id) AS total_orders,
        SUM(sale_price) AS total_revenue,
        SAFE_DIVIDE(SUM(sale_price), COUNT(DISTINCT order_id)) AS average_order_value
    FROM
        regional_data
    GROUP BY
        state
)
-- Langkah 3: Ranking negara bagian berdasarkan metrik yang telah dihitung
SELECT
    state,
    total_customers,
    total_orders,
    total_revenue,
    average_order_value,
    RANK() OVER (ORDER BY total_customers DESC) AS customer_rank,
    RANK() OVER (ORDER BY total_orders DESC) AS order_rank,
    RANK() OVER (ORDER BY total_revenue DESC) AS revenue_rank, 
    RANK() OVER (ORDER BY average_order_value DESC) AS aov_rank
FROM
    state_performance
ORDER BY
    customer_rank, order_rank, revenue_rank, aov_rank;

""")
jawaban3.head(100)


Query job started...
Waiting for query to complete...
Query completed successfully.


/home/mahaputra971/anaconda3/envs/project_digital_skola/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,state,total_customers,total_orders,total_revenue,average_order_value,customer_rank,order_rank,revenue_rank,aov_rank
0,California,1939,2531,217817.880210,86.060008,1,1,1,27
1,Texas,1339,1722,146127.410152,84.859123,2,2,2,30
2,Florida,899,1227,105070.090172,85.631695,3,3,3,28
3,New York,647,833,70648.000079,84.811525,4,4,4,31
4,Illinois,485,653,57804.980054,88.522175,5,5,5,21
5,Georgia,461,606,57458.929976,94.816716,6,6,6,7
6,Arizona,396,515,45081.750111,87.537379,7,7,7,24
7,North Carolina,384,491,41758.990101,85.048860,8,8,8,29
8,New Jersey,319,421,34844.540069,82.766128,9,9,10,35
9,Ohio,308,392,31819.970011,81.173393,10,12,14,40


In [85]:
jawaban4 = table_bigqery_to_dataframe(sql_query="""
SELECT
    EXTRACT(YEAR FROM o.created_at) AS Tahun,
    EXTRACT(QUARTER FROM o.created_at) AS Quarter,
    ROUND(SUM(oi.sale_price), 2) AS Total_Revenue,
    COUNT(DISTINCT o.user_id) AS Total_Customer,
    COUNT(oi.product_id) AS Total_Product_Terjual,
    COUNT(DISTINCT o.order_id) AS Total_Order
FROM
    bigquery-public-data.thelook_ecommerce.orders AS o
JOIN
    bigquery-public-data.thelook_ecommerce.order_items AS oi
ON
    o.order_id = oi.order_id
WHERE
    o.status = 'Complete'
GROUP BY
    tahun, quarter
ORDER BY
    tahun, quarter;
""")
jawaban4.head(100)


Query job started...
Waiting for query to complete...
Query completed successfully.


/home/mahaputra971/anaconda3/envs/project_digital_skola/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Tahun,Quarter,Total_Revenue,Total_Customer,Total_Product_Terjual,Total_Order
0,2019,1,1474.07,21,27,21
1,2019,2,5452.35,54,84,56
2,2019,3,9416.97,102,158,103
3,2019,4,14106.80,162,232,163
4,2020,1,19069.34,224,323,224
5,2020,2,21325.39,245,358,246
6,2020,3,29396.90,337,487,337
7,2020,4,31131.11,379,540,381
8,2021,1,36460.43,429,597,430
9,2021,2,41055.78,471,667,474
